# Aprendizado Supervisionado II - Trabalho 1

## Pacotes

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from collections import Counter
from sklearn import feature_extraction, model_selection, naive_bayes, metrics
import numpy as np
import sklearn.metrics as sklm
from sklearn.model_selection import train_test_split, cross_val_score
import seaborn as sns

In [ ]:
import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import warnings
warnings.filterwarnings('ignore')

In [ ]:
sms = pd.read_csv('/kaggle/input/sms-spam-collection-dataset/spam.csv')

## Análise exploratória 

In [ ]:
sms = sms[['v1', 'v2']]
sms.columns = ['class', 'text']
sms.head()

In [ ]:
sms.describe()

In [ ]:
sms.groupby('class').describe()

In [ ]:
sms.isnull().sum()

In [ ]:
sms['class'].value_counts().plot(kind = 'pie', explode = [0, 0.1], figsize = (6, 6), autopct = '%1.1f%%', shadow = True)
plt.ylabel('Spam vs Ham')
plt.legend(['Ham', 'Spam'])
plt.show()

In [ ]:
sms['class'].value_counts().plot(kind = 'bar', figsize = (5, 5))
plt.ylabel('Contagem')
plt.show()


Observamos nesses primeiros passos da análise exploratória que há um número muito superior de mensagens legítimas. Apenas 13.4% de todas as mensagens estão classificadas como "spam", enquanto 86.6% é classificada como "ham" (legítima). 


In [ ]:
sms['length']=sms['text'].apply(len)
sms.head()

In [ ]:
sms.length.describe()

In [ ]:
sms[sms['class']=='spam'].length.describe()

In [ ]:
sms[sms['class']=='ham'].length.describe()

In [ ]:
plt.figure(figsize=(12,5))
sms[sms['class']=='spam']['length'].plot(bins=35,kind='hist',color='blue',label='spam',alpha=0.5)
plt.legend()
plt.xlabel('message length')
plt.show()

In [ ]:
plt.figure(figsize=(12,5))
sms[sms['class']=='ham']['length'].plot(bins=35,kind='hist',color='red',label='ham',alpha=0.5)
plt.legend()
plt.xlabel('message length')
plt.show()

In [ ]:
sms['target']=np.where(sms['class']=='spam',1,0)

In [ ]:
spam=[]
ham=[]
spam_class=sms[sms['target']==1]['text']
ham_class=sms[sms['target']==0]['text']

In [ ]:
sns.set_style('whitegrid')

f, ax = plt.subplots(1, 2, figsize = (20, 6))

sns.distplot(sms[sms["target"] == 1]["length"], bins = 20, ax = ax[0])
ax[0].set_xlabel("Spam Message Word Length")

sns.distplot(sms[sms["target"] == 0]["length"], bins = 20, ax = ax[1])
ax[0].set_xlabel("Ham Message Word Length")

plt.show()

Aqui adicionamos uma coluna com a contagem do número de palavras em cada mensagem ao nosso data frame e conseguimos algumas informações interessantes. Temos em média aproximadamente 80 palavras por mensagem, onde a menor mensagem de 2 palavras e a maior 910. Além disso, vemos pelos gráficos que as mensagens de spam são, em média, mais longas que as mensagens legítimas.

In [ ]:
count_ham = pd.DataFrame.from_dict(Counter(' '.join(sms[sms['class'] == 'ham']['text']).lower().split()).most_common(50))
count_ham.columns = ['words in ham', 'count']
count_ham.head()

In [ ]:
count_spam = pd.DataFrame.from_dict(Counter(' '.join(sms[sms['class'] == 'spam']['text']).lower().split()).most_common(50))
count_spam.columns = ['words in spam', 'count']
count_spam.head()

In [ ]:
count_ham.plot(kind = 'bar', legend = False, figsize = (15, 3.5))
plt.xticks(np.arange(50), count_ham['words in ham'])
plt.title('Palavras mais frequentes em mensagens genuínas')
plt.xlabel('Palavras')
plt.ylabel('Contagem')
plt.show()

count_spam.plot(kind = 'bar', legend = False, color = 'orange', figsize = (15, 3.5))
plt.xticks(np.arange(50), count_spam['words in spam'])
plt.title('Palavras mais frequentes em mensagens de spam')
plt.xlabel('Palavras')
plt.ylabel('Contagem')
plt.show()


Verificamos as palavras mais comuns para cada tipo de mensagem. Para as mensagens genuínas "i" foi a palavra mais observada e para as mensagens de spam a palavra foi "to".
Essas palavras, assim como algumas das mais frequentes, são comumente classificada como "stop words" e serão removidas na etapa de pré-processamento. 

## Pré-processamento

### Removendo stopwords

In [ ]:
feat_ext = feature_extraction.text.CountVectorizer(stop_words = 'english', encoding='ansi')
stop_words = feat_ext.get_stop_words() # Stopwords

In [ ]:
words_ham = ' '.join(sms[sms['class'] == 'ham']['text']).lower().split()
words_ham_sw = [word for word in words_ham if word not in stop_words]
words_spam = ' '.join(sms[sms['class'] == 'spam']['text']).lower().split()
words_spam_sw = [word for word in words_spam if word not in stop_words]

In [ ]:
#Palavras em 'ham' sem as stopwords
count_ham_sw = pd.DataFrame.from_dict(Counter(words_ham_sw).most_common(50))
count_ham_sw.columns = ['words in ham (wo sw)', 'count']
count_ham_sw.head()

In [ ]:
#Palavras em 'spam' sem as stopwords
count_spam_sw = pd.DataFrame.from_dict(Counter(words_spam_sw).most_common(50))
count_spam_sw.columns = ['words in spam (wo sw)', 'count']
count_spam_sw.head()

In [ ]:
count_ham.plot(kind = 'bar', legend = False, figsize = (15, 3.5))
plt.xticks(np.arange(50), count_ham_sw['words in ham (wo sw)'])
plt.title('Palavras mais frequentes em mensagens genuínas (sem stop-words)')
plt.xlabel('Palavras')
plt.ylabel('Contagem')
plt.show()

count_spam.plot(kind = 'bar', legend = False, color = 'orange', figsize = (15, 3.5))
plt.xticks(np.arange(50), count_spam_sw['words in spam (wo sw)'])
plt.title('Palavras mais frequentes em mensagens de spam (sem stop-words)')
plt.xlabel('Palavras')
plt.ylabel('Contagem')
plt.show()

Ao remover as "stop words", observamos que o gráfico das palavras mais frequentes muda tanto para as mensagens de spam, quanto para as mensagens genuínas. A palavra mais comum nas mensagens genuínas passa a ser "u" e nas mensagens de spam "free". É interessante observar que ofertar algo de graça é o recurso mais usado para chamar a atenção em mensagens de spam. 

## Vetorização e matriz esparsa

### Gerando a matriz esparsa

In [ ]:
#Transformando todas as letras para minúsculas
sms['text'] = sms['text'].str.lower()

In [ ]:
X = feat_ext.fit_transform(sms['text'])
np.shape(X)

8405 atributos criados, nesse caso, palavras presentes na SMS. Armazenadas em uma matriz esparsa para economizar memória!

Atributo $j$ (coluna) na linha $i$ é igual à quantidade de vezes que a palavra associada ao índice $j$ aparece na SMS de índice $i$.

In [ ]:
# Conferindo os tamanhos...
print(np.shape(X))
print(np.shape(sms['class']))
print(np.shape(sms.index))

### Binarizando a matriz X

In [ ]:
# Entradas diferentes de zero em X
X.nonzero()

In [ ]:
# Tem entradas de fato diferentes de 0 ou 1!

np.sum(X[X.nonzero()] == 2)
#np.sum(X[X.nonzero()] > 1)

In [ ]:
X_bin = X.copy()
X_bin[X_bin.nonzero()] = 1

In [ ]:
#Matriz esparsa, sem entradas maiores do que 1
X_bin

## Dividindo em conjunto de treinamento e teste

In [ ]:
sms["class"]=sms["class"].map({'spam':1,'ham':0})
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, sms['class'], test_size=0.33, random_state=42)
print([np.shape(X_train), np.shape(X_test)])

In [ ]:
#X_bin_train, X_bin_test, y_train, y_test, idx_train, idx_test = model_selection.train_test_split(
#    X_bin, sms['class'], sms.index, test_size = 0.33)

#print([np.shape(X_bin_train), np.shape(X_bin_test)])

### Classificando


[Questão teórica discursiva] Disserte sobre qual tipo de erro é mais grave de se cometer neste problema: falso negativo ou falso positivo. Com base nisso, diga como o classificador recém construído pode ser utilizado de modo a levar tal informação em consideração; em particular, diga qual métrica de avaliação você julga que faz mais sentido de levar em consideração neste problema.

O erro mais grave de se cometer nesse problema é classificar emails legítimos como spam, isso é, obter falsos positivos. Deveríamos ter um classificador com alta precisão para que esses casos nunca entrassem no nosso modelo. Após essa condição ser cumprida, devemos escolher o modelo com a melhor acurácia.

Testaremos a acurácia de dois tipos de modelo: Bernoiulli e Multinomial.



#### Bayes Ingênuo: Multinomial

Treinaremos diferentes modelos alterando o parâmetro $\alpha$ e, posteriormente, com o conjunto de teste, utilizaremos medidas de avaliação para escolher o melhor possível.

In [ ]:
list_alpha = np.arange(1/100000, 20, 0.11)
score_train = np.zeros(len(list_alpha))
score_test = np.zeros(len(list_alpha))
recall_test = np.zeros(len(list_alpha))
precision_test= np.zeros(len(list_alpha))
count = 0
for alpha in list_alpha:
    bayes = naive_bayes.MultinomialNB(alpha=alpha)
    bayes.fit(X_train, y_train)
    score_train[count] = bayes.score(X_train, y_train)
    score_test[count]= bayes.score(X_test, y_test)
    recall_test[count] = metrics.recall_score(y_test, bayes.predict(X_test))
    precision_test[count] = metrics.precision_score(y_test, bayes.predict(X_test))
    count = count + 1 

In [ ]:
matrix = np.matrix(np.c_[list_alpha, score_train, score_test, recall_test, precision_test])
models = pd.DataFrame(data = matrix, columns = 
             ['alpha', 'Train Accuracy', 'Test Accuracy', 'Test Recall', 'Test Precision'])
models.head(n=10)

Como discutido anteriormente, precisamos obter o modelo com a maior precisão dentre os modelos com os diferentes $\alpha$. Aqui observamos que o $\alpha$ que possibilidade isso vale $15.730010$.

In [ ]:
best_index = models['Test Precision'].idxmax()
models.iloc[best_index, :]

In [ ]:
models[models['Test Precision']==1].head(n=5)

Podemos observar que vários modelos com o mesmo $\alpha$ que permitem não produzir nenhum falso positivo são possíveis. Escolheremos entre esses modelos com precisão igual a $1.0$ aquele com a maior acurácia.

In [ ]:
best_index = models[models['Test Precision']==1]['Test Accuracy'].idxmax()
bayes = naive_bayes.MultinomialNB(alpha=list_alpha[best_index])
bayes.fit(X_train, y_train)
models.iloc[best_index, :]

In [ ]:
#Matriz de confusão

m_confusion_test = metrics.confusion_matrix(y_test, bayes.predict(X_test))
pd.DataFrame(data = m_confusion_test, columns = ['Predicted 0', 'Predicted 1'],
            index = ['Actual 0', 'Actual 1'])

In [ ]:
NB_clf2 = naive_bayes.MultinomialNB(alpha=15.730010)
NB_clf2.fit(X_train, y_train)
CM2 = sklm.confusion_matrix(y_test, bayes.predict(X_test)) 

In [ ]:
sklm.plot_confusion_matrix(NB_clf2, X_test, y_test)
plt.title("Multinomial")
plt.show()

Conseguimos com que apenas 56 mensagens de spam sejam classificadas como legítimas, enquanto nenhuma mensagem legítima é perdida como spam. Além disso, tivemos uma acurácia de aproximadamente $97\%$.

#### Bayes ingênuo: Bernouilli

Utilizamos o mesmo processo, alterando o parâmetro $\alpha$ até obter a melhor precisão possível.

In [ ]:
list_alpha = np.arange(1/100000, 20, 0.11)
score_train = np.zeros(len(list_alpha))
score_test = np.zeros(len(list_alpha))
recall_test = np.zeros(len(list_alpha))
precision_test= np.zeros(len(list_alpha))
count = 0
for alpha in list_alpha:
    bayes = naive_bayes.BernoulliNB(alpha=alpha)
    bayes.fit(X_train, y_train)
    score_train[count] = bayes.score(X_train, y_train)
    score_test[count]= bayes.score(X_test, y_test)
    recall_test[count] = metrics.recall_score(y_test, bayes.predict(X_test))
    precision_test[count] = metrics.precision_score(y_test, bayes.predict(X_test))
    count = count + 1 

In [ ]:
matrix = np.matrix(np.c_[list_alpha, score_train, score_test, recall_test, precision_test])
models = pd.DataFrame(data = matrix, columns = 
             ['alpha', 'Train Accuracy', 'Test Accuracy', 'Test Recall', 'Test Precision'])
models.head(n=10)

In [ ]:
best_index = models['Test Precision'].idxmax()
models.iloc[best_index, :]

In [ ]:
models[models['Test Precision']==1].head(n=5)

In [ ]:
best_index = models[models['Test Precision']==1]['Test Accuracy'].idxmax()
bayes = naive_bayes.BernoulliNB(alpha=list_alpha[best_index])
bayes.fit(X_train, y_train)
models.iloc[best_index, :]

In [ ]:
#Confusion matrix with naive bayes classifier

m_confusion_test = metrics.confusion_matrix(y_test, bayes.predict(X_test))
pd.DataFrame(data = m_confusion_test, columns = ['Predicted 0', 'Predicted 1'],
            index = ['Actual 0', 'Actual 1'])

In [ ]:
NB_clf = naive_bayes.BernoulliNB(alpha=3.520010)
NB_clf.fit(X_train, y_train)
CM = sklm.confusion_matrix(y_test, bayes.predict(X_test)) 

In [ ]:
sklm.plot_confusion_matrix(NB_clf, X_test, y_test)
plt.title("Bernouilli")
plt.show()

Para o modelo Bernouilli, o custo para não obter falsos positivos foi detectar $243$ mensagens de spam como legítimas e apenas $9$ verdadeiros positivos. Tivemos uma acurácia de aproximadamente $87\%$.

## Curva ROC

#### Bernouilli

In [ ]:
Pop = np.size(y_test)
N = np.size(np.where(y_test == 0))
P = np.size(np.where(y_test == 1))

VN = CM[0, 0] #verdadeiro negativo
FP = CM[0, 1] #falso positivo
FN = CM[1, 0] #falso negativo
VP = CM[1, 1] #verdadeiro positivo
Prev = P/Pop
Acc = (VN + VP)/Pop

FPR = FP/N
TNR = VN/N
TPR = VP/P
FNR = FN/P

FOR = FN/(VN + FN)
PPV = VP/(FP + VP)
NPV = VN/(VN + FN)
FDR = FP/(FP + VP)

F1 = 2/(1/PPV + 1/TPR)
print('Prevalência:', Prev)
print('Acurácia:', Acc)
print('Taxa de falsos positivos:', FPR)
print('Taxa de verdadeiros negativos (Especificidade):', TNR)
print('Taxa de verdadeiros positivos (Recall):', TPR)
print('Taxa de falsos negativos:', FNR)
print('False omission rate:', FOR)
print('Valor preditivo positivo (Precisão):', PPV)
print('Valor preditivo negativo:', NPV)
print('False discovery rate:', FDR)
print('F1 Score:', F1)

In [ ]:
sklm.plot_roc_curve(NB_clf, X_test, y_test)

#### Multinomial

In [ ]:
Pop = np.size(y_test)
N = np.size(np.where(y_test == 0))
P = np.size(np.where(y_test == 1))

VN = CM2[0, 0] #verdadeiro negativo
FP = CM2[0, 1] #falso positivo
FN = CM2[1, 0] #falso negativo
VP = CM2[1, 1] #verdadeiro positivo
Prev = P/Pop
Acc = (VN + VP)/Pop

FPR = FP/N
TNR = VN/N
TPR = VP/P
FNR = FN/P

FOR = FN/(VN + FN)
PPV = VP/(FP + VP)
NPV = VN/(VN + FN)
FDR = FP/(FP + VP)

F1 = 2/(1/PPV + 1/TPR)
print('Prevalência:', Prev)
print('Acurácia:', Acc)
print('Taxa de falsos positivos:', FPR)
print('Taxa de verdadeiros negativos (Especificidade):', TNR)
print('Taxa de verdadeiros positivos (Recall):', TPR)
print('Taxa de falsos negativos:', FNR)
print('False omission rate:', FOR)
print('Valor preditivo positivo (Precisão):', PPV)
print('Valor preditivo negativo:', NPV)
print('False discovery rate:', FDR)
print('F1 Score:', F1)

In [ ]:
sklm.plot_roc_curve(NB_clf2, X_test, y_test)

## Conclusão

A análise exploratória dos dados de SMS permitiu que observassemos um desbalanceamento no número de mensagens legítimas e de spam, assim como percebessemos uma diferença significativa no número de palavras em cada tipo de texto.

No pré-processamento removemos stopwords para diminuir o espaço dimensional e transformamos todas as letras em minúsculas para não incorrer em palavras repetidas mas digitadas diferentes.

Para esse tipo de análise pareceu muito mais crítico perder mensagens legítimas ao classifica-las como spam do que o contrário, logo optamos por escolher um modelo em que a totalidade dos e-mails legítimos fosse reconhecido. Para isso buscamos obter o modelo com a maior precisão possível que eliminasse a possibilidade de falsos positivos e, entre os modelos possívels com essa precisão, aquele com a maior acurária. 

Foram sugeridos dois modelos de Bayes ingênuo: Bernoulli e Multinomial, e esses modelos apresentaram resultados bem distintos. O modelo Bernoulli para ser capaz de entregar $100\%$ dos falsos positivos apresentou uma taxa de vfalsos negativos extremamente grande. Já o modelo Multinomial foi capaz de detectar esses mesmos falsos positivos classificando apenas 56 dos 252 emails de spam como legítimos, além de apresentar uma acurácia $10\%$ maior. Apesar disso, a área sob a curva ROC é muito semelhante entre os dois modelos.

